In [1]:
# ===== Fix for np.bool / np.object error (Keep for environment compatibility) =====
import numpy as np
if not hasattr(np, 'bool'):
    np.bool = bool
if not hasattr(np, 'object'):
    np.object = object
# =================================================================================

import os
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Step 1: Install required libraries
!pip install mediapipe

# --- Data Loading and Feature Extraction ---
data_dir = "/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/"

# Dynamically get class names from folder names
classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

X = []  # Features (landmarks)
y = []  # Labels (class indices)

print("Starting feature extraction...")

for i, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    print(f"Processing class: {class_name} ({i+1}/{len(classes)})")
    
    image_files = os.listdir(class_dir)[:500]  # Use a subset of 500 images
    
    for image_file in image_files:
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)
        if image is None:
            continue
        
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = []
                for landmark in hand_landmarks.landmark:
                    landmarks.extend([landmark.x, landmark.y, landmark.z])
                X.append(landmarks)
                y.append(i)

X = np.array(X)
y = np.array(y)

print("Feature extraction completed.")
print(f"Total samples extracted: {len(X)}")

# --- Prepare Data for Training ---
lb = LabelBinarizer()
y_one_hot = lb.fit_transform(y)
num_classes = y_one_hot.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

/tmp/ipykernel_1620/1933065749.py:3: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'bool'):
/tmp/ipykernel_1620/1933065749.py:5: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'object'):
2025-09-25 02:12:12.351689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758766332.374034    1620 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758766332.380902    1620 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Starting feature extraction...
Processing class: A (1/29)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1758766338.611665    1681 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758766338.646243    1681 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758766338.655734    1679 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Processing class: B (2/29)
Processing class: C (3/29)
Processing class: D (4/29)
Processing class: E (5/29)
Processing class: F (6/29)
Processing class: G (7/29)
Processing class: H (8/29)
Processing class: I (9/29)
Processing class: J (10/29)
Processing class: K (11/29)
Processing class: L (12/29)
Processing class: M (13/29)
Processing class: N (14/29)
Processing class: O (15/29)
Processing class: P (16/29)
Processing class: Q (17/29)
Processing class: R (18/29)
Processing class: S (19/29)
Processing class: T (20/29)
Processing class: U (21/29)
Processing class: V (22/29)
Processing class: W (23/29)
Processing class: X (24/29)
Processing class: Y (25/29)
Processing class: Z (26/29)
Processing class: del (27/29)
Processing class: nothing (28/29)
Processing class: space (29/29)
Feature extraction completed.
Total samples extracted: 10648


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# --- Model Building and Training ---
print("\nBuilding and training the TensorFlow model...")
print(f"Shape of training data: {X_train.shape}")
print(f"Number of classes detected: {num_classes}")

model = Sequential([
    Dense(256, activation='relu', input_shape=(63,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nModel evaluation on test data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

# Simpan model dalam format native Keras 3 yang direkomendasikan
keras_filename = "asl_landmark_model.keras"
model.save(keras_filename)
print(f"TensorFlow model saved in native Keras 3 format to '{keras_filename}'")


Building and training the TensorFlow model...
Shape of training data: (8518, 63)
Number of classes detected: 28
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


240/240 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.1162 - loss: 3.1263 - val_accuracy: 0.5634 - val_loss: 1.7818
Epoch 2/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4881 - loss: 1.6693 - val_accuracy: 0.7852 - val_loss: 0.8991
Epoch 3/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7023 - loss: 0.9615 - val_accuracy: 0.8638 - val_loss: 0.5785
Epoch 4/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7853 - loss: 0.6999 - val_accuracy: 0.8697 - val_loss: 0.4738
Epoch 5/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8187 - loss: 0.5838 - val_accuracy: 0.9120 - val_loss: 0.3796
Epoch 6/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8480 - loss: 0.5095 - val_accuracy: 0.9190 - val_loss: 0.3341
Epoch 7/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8637 - loss: 0.4391 - val_accuracy: 0.8932 - val_loss: 0.3408
Epoch 8/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8824 - loss: 0.3734 - val_accuracy: 0.9425 - val_

In [19]:
import tf2onnx
import onnx
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout
import os 
import numpy as np

# Pastikan library sudah terinstal
!pip install tf2onnx onnx

# --- ONNX Conversion ---
print("\nConverting Model to ONNX...")

keras_filename = "asl_landmark_model.keras" 
onnx_filename = "asl_landmark_model.onnx"
input_dim = 63 

try:
    # 1. Muat model Keras
    if not os.path.exists(keras_filename):
        raise FileNotFoundError(f"File model Keras tidak ditemukan: {keras_filename}. Pastikan Anda sudah melatih dan menyimpan modelnya.")
        
    sequential_model = load_model(keras_filename)
    
    # 💥 PERBAIKAN: Gunakan .units untuk mendapatkan jumlah output dari lapisan Dense terakhir.
    # [:-1] mengambil lapisan Dense terakhir dari model Sequential
    num_classes = sequential_model.layers[-1].units 
    
    # 2. Buat ulang model sebagai Functional API Model
    print(f"Model memiliki {num_classes} kelas. Merekonstruksi sebagai Functional API Model...")
    
    input_tensor = Input(shape=(input_dim,), name='input_features')
    # Mereplikasi arsitektur asli
    x = Dense(256, activation='relu', name='dense_1')(input_tensor)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu', name='dense_2')(x)
    x = Dropout(0.3)(x)
    output_tensor = Dense(num_classes, activation='softmax', name='output_softmax')(x)
    
    functional_model = Model(inputs=input_tensor, outputs=output_tensor)

    # 3. Transfer bobot dari model Sequential yang dilatih ke Functional Model
    functional_model.set_weights(sequential_model.get_weights())
    
    # 4. Definisikan Input Signature
    input_signature = [
        tf.TensorSpec([None, input_dim], tf.float32, name='input_features')
    ]
    
    # 5. Konversi menggunakan fungsi from_keras
    onnx_model, _ = tf2onnx.convert.from_keras(
        functional_model, 
        input_signature=input_signature,
        output_path=onnx_filename,
        opset=13,
    )
    
    print(f"Model successfully converted and saved as '{onnx_filename}'")
    
except Exception as e:
    print(f"Conversion failed: {e}")


Converting Model to ONNX...
Model memiliki 28 kelas. Merekonstruksi sebagai Functional API Model...
Model successfully converted and saved as 'asl_landmark_model.onnx'


I0000 00:00:1758767328.107544    1620 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
I0000 00:00:1758767328.107746    1620 single_machine.cc:361] Starting new session
I0000 00:00:1758767328.109917    1620 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758767328.110111    1620 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
I0000 00:00:1758767328.136227    1620 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758767328.136480    1620 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 M